In [ ]:
import streamlit as st
import pandas as pd
import numpy as np
#import plotly as px
#import plotly.figure_factory as ff
#from bokeh.plotting import figure
import matplotlib.pyplot as plt


In [ ]:

st.image("http://www.usfirepolice.net/ca_california/ca_san_francisco_county_1/wpd4c7532d_06.png", width=180)

st.markdown("<h1 style='text-align: center; color: #FF0000;'>POLICE INCIDENT REPORTS FROM 2018 TO 2020 IN SAN FRANCISCO</h1>", unsafe_allow_html=True)

df= pd.read_csv("Police_Department_Incident_Reports__2018_to_Present.csv")

st.markdown("The data shown below belongs to incident reports in the city of San Francisco, from the year 2018 to 2020, with details from each case such as date, day of the week, police district, neighborhood in which it happened, type of incident in category and subcategory, exact location and resolution.")

mapa=pd.DataFrame()
mapa["Date"]=df["Incident Date"]
mapa["Day"]=df["Incident Day of Week"]
mapa["Police District"]=df["Police District"]
mapa["Neighborhood"]=df["Analysis Neighborhood"]
mapa["Incident Category"]=df["Incident Category"]
mapa["Incident Subcategory"]=df["Incident Subcategory"]
mapa["Resolution"]=df["Resolution"]
mapa["lat"]=df["Latitude"]
mapa["lon"]=df["Longitude"]
mapa=mapa.dropna()


subset_data2 = mapa
police_district_input = st.sidebar.multiselect(
'Police District',
mapa.groupby('Police District').count().reset_index()['Police District'].tolist())
if len(police_district_input) > 0:
    subset_data2 = mapa[mapa['Police District'].isin(police_district_input)]

subset_data1 = subset_data2
neighborhood_input = st.sidebar.multiselect(
'Neighborhood',
subset_data2.groupby('Neighborhood').count().reset_index()['Neighborhood'].tolist())
if len(neighborhood_input) > 0:
    subset_data1 = subset_data2[subset_data2['Neighborhood'].isin(neighborhood_input)]

subset_data = subset_data1
incident_input = st.sidebar.multiselect(
'Incident Category',
subset_data1.groupby('Incident Category').count().reset_index()['Incident Category'].tolist())
if len(incident_input) > 0:
    subset_data = subset_data1[subset_data1['Incident Category'].isin(incident_input)]

#FILTRO
years = pd.to_datetime(subset_data['Date']).dt.year.unique()
# Crear la barra lateral con opciones de filtrado por año
year_input = st.sidebar.multiselect('Year', years)
# Filtrar los datos por el año seleccionado
if len(year_input) > 0:
    subset_data = subset_data[pd.to_datetime(subset_data['Date']).dt.year.isin(year_input)]


#FILTRO
# Obtener los valores únicos de la columna "Resolution"
resolution_values = subset_data['Resolution'].unique()
# Crear la barra lateral con opciones de filtrado por resolución
resolution_input = st.sidebar.multiselect('Resolution', resolution_values)
# Filtrar los datos por la resolución seleccionada
if len(resolution_input) > 0:
    subset_data3 = subset_data[subset_data['Resolution'].isin(resolution_input)]


subset_data

st.markdown('It is important to mention that any police district can answer to any incident, the neighborhood in which it happened is not related to the police district.')
st.markdown('Crime locations in San Francisco')
st.map(subset_data)
st.markdown("Crimes ocurred per day of the week")
st.bar_chart(subset_data['Day'].value_counts ())
st.markdown("Crimes ocurred per date")
st.line_chart(subset_data['Date'].value_counts ())
st.markdown('Type of crimes committed')
st.bar_chart(subset_data['Incident Category'].value_counts())

st.markdown("Incident Subcategory")
st.bar_chart(subset_data['Incident Subcategory'].value_counts ())

agree = st.button('click to see Incident Subcategories')
if agree:
    st.markdown("Subtype of crimes committed")
    st.bar_chart(subset_data['Incident Subcategory'].value_counts ())


st.markdown("Resolution status")
fig1, ax1 = plt.subplots ()
labels = subset_data ['Resolution'].unique()
ax1.pie(subset_data['Resolution'].value_counts (), labels=labels, autopct='%1.1f%%', startangle=20)
st.pyplot(fig1)


st.markdown("Bunch of day ocurred")
fig2, ax2 = plt.subplots ()
labels = subset_data ['Day'].unique()
ax2.pie(subset_data['Day'].value_counts (), labels=labels, autopct='%1.1f%%', startangle=20)
st.pyplot(fig2)


#XXXXXXXXXXXXX
# Obtener el conteo de crímenes por distrito policial
crime_count = subset_data['Police District'].value_counts()

# Configurar la paleta de colores en tonos rojos oscuros
colors = ['#990000', '#CC0000', '#FF0000', '#FF3333', '#FF6666', '#FF9999']

# Crear la gráfica de barras
fig, ax = plt.subplots()
bars = ax.bar(crime_count.index, crime_count.values, color=colors)

# Obtener los valores máximos y mínimos del conteo de crímenes
max_count = max(crime_count.values)
min_count = min(crime_count.values)

# Añadir el indicador de escala de color
for bar in bars:
    value = bar.get_height()
    normalized_value = (value - min_count) / (max_count - min_count)
    color_index = int((1 - normalized_value) * (len(colors) - 1))  # Invertir el índice del color
    bar.set_color(colors[color_index])

# Configurar el estilo de la gráfica
ax.set_xlabel('Police District')
ax.set_ylabel('Count')
ax.set_title('Crime Count by Police District')

# Rotar las etiquetas del eje x para una mejor legibilidad
plt.xticks(rotation=45)

# Mostrar la gráfica en Streamlit
st.markdown("Crime Count by Police District")
st.pyplot(fig)


# Obtener el conteo de registros por año
subset_data['Year'] = pd.to_datetime(subset_data['Date']).dt.year
records_per_year = subset_data['Year'].value_counts().sort_index()

# Configurar la paleta de colores en tonos rojos oscuros
colors = ['#990000', '#CC0000', '#FF0000']

# Crear la gráfica de barras
fig, ax = plt.subplots()
bars = ax.bar(records_per_year.index.astype(str), records_per_year.values, color=colors)

# Obtener los valores máximos y mínimos del conteo de registros
max_count = max(records_per_year.values)
min_count = min(records_per_year.values)

# Añadir el indicador de escala de color
for bar in bars:
    value = bar.get_height()
    normalized_value = (value - min_count) / (max_count - min_count)
    color_index = int((1 - normalized_value) * (len(colors) - 1))  # Invertir el índice del color
    bar.set_color(colors[color_index])

# Configurar el estilo de la gráfica
ax.set_xlabel('Year')
ax.set_ylabel('Count')
ax.set_title('Total Records by Year')

# Mostrar la gráfica en Streamlit
st.markdown("Total Records by Year")
st.pyplot(fig)
